# ANÁLISE SISTEMA DE INVESTIMENTOS EM BOLSA DURANTE A PANDEMIA DE 2020 (IFR - INDICE DE FORÇA RELATIVA / RSI - RELATIVE STRENGTH INDEX)

<b>Introdução<b>

Buscamos neste projeto analisar e comparar as melhores taxas de retorno, desempenho e drawdown apenas de ações que possuem bom desempenho estatistico em um sistema de investimentos de ações na bolsa de valores brasileira, utilizado com frequência em mercados globais e baseado em um indicador de volatilidade. Este sistema utiliza o indicador IFR2-Indice de Força Relativa no período 2 (RSI – Relative Strength Index) para monitorar e dar sinais de compra e venda de ações considerando apenas o gráfico diário dos ativos.

<b>Problema a ser solucionado<b>

Como foi o desempenho e resultados dos ativos selecionados – considerando taxa de retorno e drawdown como parâmetros principais – em um cenário atípico de pandemia global (popularmente chamado de “Cisne Negro” nos mercados financeiros) ocorrido em 2020?

<b>Metodologia utilizada<b>

- Escolha do sistema:  1 sistema objetivo utilizando o indicador IFR período 2 
- Testes estatísticos (backtests) do sistema aplicado as ações da bolsa de valores brasileira utilizando a linguagem de programação NSTL e outro software sobressalente para confirmação da acurácia dos dados e resultados obtidos.
- Análise de métricas e performance das ações no sistema objetivo no intervalo estipulado
- Classificação de ações com melhores e piores desempenhos para formação do dataset a ser utilizado

--

- Python:
1. Formação de carteira teórica de ações composta pelos ativos que ofereceram melhor desempenho individual no sistema (“Carteira IFR2”). 
2. Backtesting da carteira no mesmo intervalo estipulado (2020) utilizando a linguagem de programação python e tendo o Ibovespa como benchmark. 
3. Resultados obtidos: vemos se a carteira nos forneceu retornos percentuais e taxa de drawdown favoraveis no prazo estabelecido, respondendo ao nosso questionamento inicial.

In [9]:
#Importando bibliotecas para analises
import pandas as pd
from pandas_datareader import data as wb
import pyfolio as pf
import matplotlib.pyplot as plt
%matplotlib inline
import bt
import seaborn as sns

In [10]:
#Carregar dados: fechamentos, retorno e retorno acumulado
carteiraIFR2 = ['LREN3.SA','MGLU3.SA', 'CPLE6.SA', 'EZTC3.SA', 'VIVT3.SA', 'TAEE11.SA', '^BVSP']
grupo_ativosIFR2=pd.DataFrame()
for t in carteiraIFR2:
  grupo_ativosIFR2[t]=wb.DataReader(t, data_source='yahoo', start='01-01-2020', end='12-31-2020')['Adj Close']
grupo_ativosIFR2.head()

,LREN3.SA,MGLU3.SA,CPLE6.SA,EZTC3.SA,VIVT3.SA,TAEE11.SA,^BVSP
Date,,,,,,,
2020-01-02,56.448383,12.038459,6.588762,52.364841,43.942810,27.722883,118573.0
2020-01-03,56.055695,11.931082,6.583965,52.781143,44.452923,27.686924,117707.0
2020-01-06,56.006611,11.940844,6.594518,53.028938,44.024796,27.857721,116878.0
2020-01-07,55.977158,11.906678,6.628095,53.762424,44.999477,27.525120,116662.0
2020-01-08,56.340397,12.272737,6.524486,54.317493,45.454945,27.246450,116247.0


In [11]:
grupo_ativosIFR2.rename(columns={'^BVSP': 'IBOVESPA'}, inplace=True)

In [12]:
retorno = grupo_ativosIFR2.pct_change()
retorno.head(3)

,LREN3.SA,MGLU3.SA,CPLE6.SA,EZTC3.SA,VIVT3.SA,TAEE11.SA,IBOVESPA
Date,,,,,,,
2020-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,-0.006957,-0.008920,-0.000728,0.007950,0.011609,-0.001297,-0.007304
2020-01-06,-0.000876,0.000818,0.001603,0.004695,-0.009631,0.006169,-0.007043


In [13]:
retorno_acumuladoIFR2=(1+retorno).cumprod()
retorno_acumuladoIFR2.iloc[0]=1
retorno_acumuladoIFR2.head(3)

,LREN3.SA,MGLU3.SA,CPLE6.SA,EZTC3.SA,VIVT3.SA,TAEE11.SA,IBOVESPA
Date,,,,,,,
2020-01-02,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2020-01-03,0.993043,0.991080,0.999272,1.007950,1.011609,0.998703,0.992696
2020-01-06,0.992174,0.991891,1.000874,1.012682,1.001866,1.004864,0.985705


In [14]:
retorno_acumuladoIFR2.plot(figsize=(16,8), title='Carteira IFR2 - Desempenho Papeis solo 2020')

<AxesSubplot:title={'center':'Carteira IFR2 - Desempenho Papeis solo 2020'}, xlabel='Date'>

CRIANDO CARTEIRA TEORICA E APLICANDO BACKTEST

In [15]:
#Criando carteira teorica e seu valor para backtest
carteira=10000*retorno_acumuladoIFR2.iloc[:, :10]
carteira["saldo"]=carteira.sum(axis=1)
carteira["retorno"]=carteira["saldo"].pct_change()
carteira

,LREN3.SA,MGLU3.SA,CPLE6.SA,EZTC3.SA,VIVT3.SA,TAEE11.SA,IBOVESPA,saldo,retorno
Date,,,,,,,,,
2020-01-02,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,70000.000000,NaN
2020-01-03,9930.434013,9910.804984,9992.719449,10079.500236,10116.085603,9987.029039,9926.964823,69943.538149,-0.000807
2020-01-06,9921.738687,9918.913838,10008.735937,10126.821129,10018.657312,10048.637836,9857.050087,69900.554827,-0.000615
2020-01-07,9916.520951,9890.533642,10059.696898,10266.893390,10240.464214,9928.664186,9838.833461,70141.606742,0.003448
2020-01-08,9980.869877,10194.607739,9902.445686,10372.893706,10344.114213,9828.144571,9803.833925,70426.909717,0.004068
...,...,...,...,...,...,...,...,...,...
2020-12-22,7645.923107,20877.941074,10531.449808,8230.713065,10114.235672,11889.094946,9812.351884,79101.709556,0.002402
2020-12-23,7745.129026,20720.277894,10848.864451,8114.223302,10136.519059,11885.488423,9939.615258,79390.117413,0.003646
2020-12-28,7789.417274,20952.623966,11102.213143,8173.422862,10227.879296,12130.773266,10040.312719,80416.642526,0.012930


In [16]:
pf.create_simple_tear_sheet(returns=(carteira['retorno']))

Start date,2020-01-02
End date,2020-12-30
Total months,11
,Backtest
Annual return,15.6%
Cumulative returns,15.3%
Annual volatility,42.8%
Sharpe ratio,0.56
Calmar ratio,0.39
Stability,0.25
Max drawdown,-39.6%
